In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
from matplotlib import pyplot as plt
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *
from eval_utils import *
from sort import *
from VehicleDetection import *

In [2]:
# Load GT detections
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)

# Load computed detections
detection_path = 'retinanet_101_detections_trained.pkl'
data_path = '../../AICity_data/train/S03/c010/'

with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

# Load video frames
video_path = '../../AICity_data/train/S03/c010/vdo.avi' #UPDATE PATH WHEN PUSHING
vidcap = cv2.VideoCapture(video_path)
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

## Overlapping boxes approach

In [3]:
# Select optical flow compensation
    # No correction: None
    # Homemade block matching [forward]: 'bm'
    
OF = 'bm'

In [4]:
#     # If frame images "imgs" are provided, use optical flow approach to improve tracking
#     if imgs is not None:
#         idxs = detections_pd.index[detections_pd['updated'] == False].tolist() #Non-updated detections
#         for ind in idxs:
#             if detections_pd.at[ind, 'opt_flow'] < 1: # After _ consecutive frames using OF, remove detection
#                 box = detections_pd.at[ind, 'bbox']
#                 block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]

#                 #Compute optical flow of the not updated tracking bounding box
#                 of = compute_block_of(block, box, imgs[1], 'backward', 10)
                
#                 #Update bounding box
#                 if len(of) > 0:
#                     if (of[0] != 0.0 or of[1] != 0.0):
#                         newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
#                         detections_pd.at[ind, 'bbox'] = newBox
#                         detections_pd.at[ind, 'updated'] = True # NEED TO DOUBLE CHECK THIS
#                         detections_pd.at[ind, 'detection'].updateBBox(newBox)
#                         detections_pd.at[ind, 'line'].append(get_box_center(newBox))
#                         detections_pd.at[ind, 'opt_flow'] = detections_pd.at[ind, 'opt_flow'] + 1

In [5]:
# def compute_block_of(block, box, im_target, mode, P, stride=1):
#     blocH_size = block.shape[0]
#     blocW_size = block.shape[1]                
#     area_minx = max(0, int(box[0]) - P)
#     area_miny = max(0, int(box[1]) - P)
#     area_maxX = min(im_target.shape[1], int(box[2]) + P)
#     area_maxy = min(im_target.shape[0], int(box[3]) + P)
#     minDist = inf
#     of = []
#     for y in (range(area_miny, area_maxy-blocH_size, stride)):
#         for x in range(area_minx, area_maxX-blocW_size, stride):
#             dist = distance(block, im_target[y:y+blocH_size, x:x+blocW_size], 'ssd')
#             if dist < minDist:
#                 minDist = dist
#                 if mode == 'forward':
#                     of = np.array([box[0] - x, box[1] - y])
#                 elif mode == 'backward':
#                     of = np.array([x - box[0], y - box[1]])
#     return of

In [4]:
def update_track3(detections_pd, previous_detections_pd, tolerance = 0.5, of = None, imgs = None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)

    previous_tracks = []

    for index, previous_detection in previous_detections_pd.iterrows():
    
        # Calculating IoUs
        IoUlist = []

        ###############

        # Apply OF correction if applicable
        newBox = apply_of_to_box(previous_detection['bbox'], [imgs[0], imgs[1]])

        previous_detection['detection'].updateBBox(newBox)

        # if OF == 'bm':
        #     box = previous_detection['bbox']
        #     block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]

        #     of = compute_block_of(block, box, imgs[1], 'backward', 10, stride = 2)

        #     if len(of) > 0:
        #         if (of[0] != 0.0 or of[1] != 0.0):
        #             newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
        #             previous_detection['detection'].updateBBox(newBox)


        ##############
        
        length, _ = detections_pd.shape

        for i in range(length):
            IoU = previous_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)

        indexMax = IoUlist.index(max(IoUlist))

        # Updating detection tracks based on best IoU matches
        if max(IoUlist) > tolerance and detections_pd.at[indexMax, 'updated'] != True:
            detections_pd.at[indexMax, 'track'] = previous_detection['track']
            detections_pd.at[indexMax, 'colour'] = previous_detection['colour']
            for line in previous_detection['line']:
                if line != detections_pd.at[indexMax,'line'][0]:
                    detections_pd.at[indexMax,'line'].append(line)
            detections_pd.at[indexMax, 'updated'] = True

        previous_tracks.append(previous_detection['track'])

    # Create new tracks for unmatched detections
    idxs = detections_pd.index[detections_pd['updated'] == False].tolist()
    track_check = 1
    for ind in idxs:
        while track_check in previous_tracks:
            track_check += 1
        detections_pd.at[ind, 'track'] = track_check
        previous_tracks.append(track_check)
            
    return detections_pd

In [5]:
detection_history = []

# Get the first frame
detections_prev_pd = get_detection_dataframe(all_detections['0'], firstFrame = True)

# Pre-process the first frame if needed
detections_prev_pd = remove_overlaps(detections_prev_pd, 0.9)
detection_history.append(detections_prev_pd)

vidcap = cv2.VideoCapture(video_path)
_, past_frame = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, current_frame = vidcap.read()
    
    detections_pd = get_detection_dataframe(all_detections[str(frame)])
    detections_pd = remove_overlaps(detections_pd, 0.9)
    detections_pd = update_track3(detections_pd, detections_prev_pd, tolerance=0.5, of = OF, imgs=[past_frame, current_frame])
    detection_history.append(detections_pd)
    detections_prev_pd = detections_pd

    past_frame = current_frame

pkl.dump(detection_history, open(f'tracking_history_{str(OF)}.pkl', 'wb'))

100%|██████████| 2139/2139 [40:07<00:00,  1.13s/it]


In [6]:
detections = {}
with open(f'tracking_history_{str(OF)}.pkl','rb') as openFile:
    detections = pkl.load(openFile)

## Animation generation

In [7]:
# Generating a tracking GIF for a frame interval
gif1 = True
initial_gif_frame = 750
final_gif_frame = 820
scaling = 0.5

ims = []

vidcap = cv2.VideoCapture(video_path)
vidcap.set(1,initial_gif_frame)

if gif1:
    _, initial_image = vidcap.read()

    for i in tqdm(range(initial_gif_frame, final_gif_frame)):
        _, image = vidcap.read()
        for index, row in detections[i].iterrows():
            image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'], colour=row['colour'])
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)

        ims.append(image)

    imageio.mimsave('tracking.gif', ims, fps=20, duration = 0.1)

In [3]:
# Generating a GIF with current and previous detections
gif2 = True
initial_gif_frame = 600
final_gif_frame = 650
scaling = 0.5

ims = []

vidcap = cv2.VideoCapture(video_path)
vidcap.set(1,initial_gif_frame)

if gif2:
    _, initial_image = vidcap.read()

    for i in tqdm(range(initial_gif_frame, final_gif_frame)):
        _, image = vidcap.read()

        detections_pd = get_detection_dataframe(all_detections[str(i)])
        detections_pd_prev = get_detection_dataframe(all_detections[str(i-1)])

        # Plot previous detections in red
        for index, row in detections_pd_prev.iterrows():
            image = drawTrackingOnImage(image, row['bbox'], colour=(0, 0, 255), showTracking = False)

        # Plot current detections in blue
        for index, row in detections_pd.iterrows():
            image = drawTrackingOnImage(image, row['bbox'], colour=(255, 0, 0), showTracking = False)
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)

        ims.append(image)

    imageio.mimsave('current_and_previous_detections.gif', ims, fps=20, duration = 0.2)

100%|██████████| 50/50 [00:01<00:00, 48.45it/s]


In [4]:
# Generating a GIF with current and OF-displaced previous detections
gif3 = True
initial_gif_frame = 600
final_gif_frame = 650
scaling = 0.5

ims = []

vidcap = cv2.VideoCapture(video_path)
vidcap.set(1,initial_gif_frame)

if gif3:
    _, initial_image = vidcap.read()

    for i in tqdm(range(initial_gif_frame, final_gif_frame)):
        _, image = vidcap.read()

        detections_pd = get_detection_dataframe(all_detections[str(i)])
        detections_pd_prev = get_detection_dataframe(all_detections[str(i-1)])

        # Plot previous detections in green
        for index, row in detections_pd_prev.iterrows():
            correctedBox = apply_of_to_box(row['bbox'], [initial_image, image])

            image_edit = drawTrackingOnImage(image, correctedBox, colour=(0, 255, 0), showTracking = False)

        # Plot current detections in blue
        for index, row in detections_pd.iterrows():
            image = drawTrackingOnImage(image, row['bbox'], colour=(255, 0, 0), showTracking = False)

        initial_image = image

        image_edit = cv2.cvtColor(image_edit, cv2.COLOR_BGR2RGB)
        image_edit = cv2.resize(image_edit, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)

        ims.append(image_edit)

    imageio.mimsave('current_and_previous_corrected_detections.gif', ims, fps=20, duration = 0.2)

100%|██████████| 50/50 [00:56<00:00,  1.12s/it]


In [12]:
vidcap = cv2.VideoCapture(video_path)
_, past_frame = vidcap.read()

for frame in tqdm(range(1,num_frames-1)):
    _, current_frame = vidcap.read()

100%|██████████| 2139/2139 [00:10<00:00, 208.21it/s]


## Evaluation

In [13]:
total_frames = 2141
initial_frame = 535

In [15]:
acc = create_accumulator()

for frame in tqdm(range(initial_frame, total_frames-1)):
    acc, frame_id = tracking_acc(frame, gt_detect, detections, acc)

display_metrics(acc)

100%|██████████| 1605/1605 [01:17<00:00, 20.65it/s]


     num_frames  precision  recall      idp       idr      idf1
acc        1605   0.593536     1.0  0.56759  0.956287  0.712366
